## Goals

- Open a roman science file with the python asdf library
- Use tools for looking at the file and searching for attributes
- Display an image contained in the file
- Access and modify metadata
- Write the file to disk

The file roman.asdf is presumed to be in the tutorial's data/ directory. This is simulated data from the [OpenUniverse2024 project](https://irsa.ipac.caltech.edu/data/theory/openuniverse2024/overview.html) packaged as a level 2 calibrated data model

In [ ]:
import numpy as np
import asdf

from matplotlib import pyplot as plt
%matplotlib widget

Load ASDF file
--------------

In [ ]:
af = asdf.open('../data/roman.asdf')

The file can be rendered using the **info** method

In [ ]:
af.info()

In [ ]:
# The file has lots of content so some options are needed to see it all

af.info(max_rows=200)

**info** is also a utility function. If called like that it can render specific nodes in the tree.

And there's a command line utility **asdftool info** which allows looking at files outside a Python session.

In [ ]:
asdf.info(af['roman']['meta']['instrument'])

Another useful utility is **search** - search by key, value or type.

In [ ]:
# Search for attributes with exposure in their name
af.search('exposure')

##### Accessing attributes 

Attributes are accessed using a dict-like syntax.

In [ ]:
# Accessing attributes: assign an attribute value to a variable
exposure_time = af.tree['roman']['meta']['exposure']['exposure_time']
print(exposure_time)

In [ ]:
# Now load image and display it
im = af.tree['roman']['data']

In [ ]:
# Obviously we need to adjust the stretch on this. I've already found a reasonable one.
fig, ax = plt.subplots()
ax.imshow(im, vmin=18, vmax=200, origin='lower')

In [ ]:
# To use a log transfer function means getting rid of all NaNs
nim = np.array(im) # Existing image is only read only and not a numpy array
nim[np.isnan(nim)] = np.nanmax(nim)
nim -= nim.min() - 1
fig1, ax1 = plt.subplots()
ax1.imshow(np.log(nim), origin='lower')

Modifying the contents
----------------------

In [ ]:
# Arbitrarily change the exposure_time
af.tree['roman']['meta']['exposure']['exposure_time'] = 1800
print(af.tree['roman']['meta']['exposure']['exposure_time'])

In [ ]:
# Replace data attribute with the log version of the image.
af.tree['data'] = np.log(nim)

In [ ]:
# Create new meta attribute
af.tree['inspected_by'] = 'Jacques Clouseau'

Saving the modified contents
----------------------------

In [ ]:
# Save to a different file
af.write_to('roman_modified.asdf')

In [ ]:
ls

## Roman datamodels
Roman ASDF files can also be opened with roman_datamodels which provides a slightly different API (and is used by the romancal calibration pipeline).

In [ ]:
import roman_datamodels
model = roman_datamodels.open("../data/roman.asdf")
print(model)

The major differences with the datamodel and "vanilla" asdf API are:
- nesting uses attribute instead of dictionary access
- there is no top level "roman" key/attribute
- saving uses "save" instead of "writeto"

In [ ]:
model.meta.exposure.exposure_time

In [ ]:
model.save("new_file.asdf")

Exercises
---------

1. Search for the name of the PI ("Klaus") and replace the attribute containing
   it with your own name. Type "af.search?" to show how to search values instead of attribute names.
3. Use the above log image and set all pixels in rows 3000:3010 and columns 4000:4010
   to a value of 2.5, and replace the data attribute with the modified log image.
5. Save the result to a new file named "roman_exercise.asdf"
6. Load that new file and display the data image
7. Print out the value of the attribute that contains the name of the PI
   to confirm the edits were saved